<a href="https://colab.research.google.com/github/juanrueda5/Magnetic-Domains-With-Machine-Learning/blob/main/SNNTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snntorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 3.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import snntorch as snn
import torch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import cv2
import io
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
import zipfile
import PIL

In [ ]:

import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
from tqdm import tqdm
cudnn.benchmark = True

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import time
import os
from tempfile import TemporaryDirectory

In [ ]:
num_steps = 100
batch_size =128
num_epochs = 15

dtype = torch.float# Torch Variables

# **Define Device**

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# **Transformations**

In [ ]:
data_transforms = {
    'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Resize((256,256)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))
            ]),
    'val': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Resize((256,256)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))
            ]),
}

# **Read Images on ZipFiles**

In [ ]:
class ZipDataset():
    def __init__(self, root_path, var,trns, cache_into_memory=False):
        if cache_into_memory:
            f = open(root_path, 'rb')
            self.zip_content = f.read()
            f.close()
            self.zip_file = zipfile.ZipFile(io.BytesIO(self.zip_content), 'r')
        else:
            self.zip_file = zipfile.ZipFile(root_path, 'r')
        self.name_list = list(filter(lambda x: var in x, self.zip_file.namelist()))
        self.to_tensor = ToTensor()
        self.transf = trns[var]
    def __getitem__(self, key):
        buf = self.zip_file.read(name=self.name_list[key])
        img = cv2.imdecode(np.frombuffer(buf, dtype=np.uint8), cv2.IMREAD_COLOR)
        img = PIL.Image.fromarray(img)
        values = self.name_list[key].split('_')
        Ms = float(values[0].split('/')[-1])
        Aex = float(values[1])
        keff = float(values[2])
        alpha = float(values[3])
        maxMs = 1900000.0
        maxAex = 2.4999999999999998e-11
        maxkeff = 1000000.0
        maxalpha = 0.95
        target = [Ms, Aex, keff, alpha]
        target = torch.tensor(target, dtype=torch.float32)
        img = self.transf(img)
        return img, target

    def __len__(self):
        return len(self.name_list)

In [ ]:
path1= '/content/drive/MyDrive/FinalDataMD/CroppedImages.zip'
ZipDataset(root_path=path1, var='train', trns=data_transforms).__getitem__(0)

(tensor([[[0.0392, 0.0275, 0.0235,  ..., 0.8510, 0.8627, 0.8824],
          [0.0353, 0.0235, 0.0235,  ..., 0.8588, 0.8471, 0.8588],
          [0.0353, 0.0353, 0.0353,  ..., 0.8745, 0.8471, 0.8471],
          ...,
          [0.0275, 0.0392, 0.0431,  ..., 0.0275, 0.0235, 0.0235],
          [0.0745, 0.0863, 0.0941,  ..., 0.0275, 0.0235, 0.0275],
          [0.2392, 0.2471, 0.2706,  ..., 0.0314, 0.0314, 0.0314]]]),
 tensor([1.9000e+06, 2.5000e-11, 4.0000e+05, 7.5000e-01]))

In [ ]:
dataset_train = ZipDataset(root_path = path1,var='train', trns = data_transforms, cache_into_memory=False)
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=16, shuffle=True)
dataset_val = ZipDataset(path1,var='val',trns = data_transforms,cache_into_memory=False)
dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=16, shuffle=True)
image_datasets = {'train': dataset_train, 'val': dataset_val}
dataloaders = {'train': dataloader_train, 'val': dataloader_val}
dataset_sizes = {'train': len(image_datasets['train']),'val': len(image_datasets['val'])}
print(dataset_sizes)

{'train': 203, 'val': 53}


# **Surrogate Function**

In [ ]:
from snntorch import surrogate
spike_grad = surrogate.fast_sigmoid(slope=25)
beta = 0.5
num_steps = 50


In [ ]:
net = nn.Sequential(nn.Conv2d(1, 32, 3),
                    nn.MaxPool2d(2),
                    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
                    nn.Conv2d(32, 64, 3),
                    nn.MaxPool2d(2),
                    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
                    nn.Conv2d(64, 128, 3),
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(),
                    nn.Linear(128, 256),
                    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
                    nn.Linear(256,128),
                    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
                    nn.Linear(128,64),
                    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
                    nn.Linear(64,4),
                    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True),
                    ).to(device)

In [ ]:
from snntorch import utils
def forward_pass(net, num_steps, data):
  mem_rec = []
  spk_rec = []
  utils.reset(net)
  #forward pass
  for step in range(num_steps):
      spk_out, mem_out = net(data)
      spk_rec.append(spk_out)
      mem_rec.append(mem_out)
  spk_tensor = torch.stack(spk_rec,dim = 0)
  mem_tensor = torch.stack(mem_rec, dim = 0)
  #compute mean
  spk_avg = torch.mean(spk_tensor, dim=0)
  mem_avg = torch.mean(mem_tensor, dim=0)

  return spk_tensor, mem_tensor


In [ ]:
def acc_rate(spk_out, targets):
   idx = spk_out.sum(dim=0)
   accuracy = np.mean((targets == idx).detach().cpu().numpy())
   return accuracy

# **Batch Accuracy**

In [ ]:
def batch_accuracy(train_loader, net, num_steps):
  with torch.no_grad():
    total = 0
    acc = 0
    net.eval()

    train_loader = iter(train_loader)
    for data, targets in train_loader:
      data = data.to(device)
      targets = targets.to(device)
      spk_rec, _ = forward_pass(net, num_steps, data)
      acc += acc_rate(spk_rec, targets) * spk_rec.size(1)
      total += spk_rec.size(1)

  return acc/total

In [ ]:
for name, param in net.named_parameters():
    print(f"{name} is on {param.device}")

0.weight is on cuda:0
0.bias is on cuda:0
3.weight is on cuda:0
3.bias is on cuda:0
6.weight is on cuda:0
6.bias is on cuda:0
9.weight is on cuda:0
9.bias is on cuda:0
11.weight is on cuda:0
11.bias is on cuda:0
13.weight is on cuda:0
13.bias is on cuda:0
15.weight is on cuda:0
15.bias is on cuda:0


# **Train Loop**

In [ ]:
import snntorch.functional as SF

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-2, betas=(0.9, 0.999))
num_epochs = 1
loss_hist = []
test_acc_hist = []
counter = 0
train_loader = dataloaders['train']
test_loader = dataloaders['val']
net.to(device)
# Outer training loop
for epoch in range(num_epochs):

    # Training loop
    for data, targets in iter(train_loader):
        data = data.to(device)
        targets = targets.to(device)
        # forward pass
        net.train()

        spk_rec, mem_rec = forward_pass(net, num_steps, data)
        print(spk_rec.shape)
        spk_avg = torch.mean(spk_rec, dim=0)
        mem_avg = torch.mean(mem_rec, dim=0)
        # initialize the loss & sum over time
        loss_val = loss_fn(spk_avg, targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward(retain_graph = True)
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())

        # Test set
        if counter % 50 == 0:
          with torch.no_grad():
            net.eval()

            # Test set forward pass
            test_acc = batch_accuracy(train_loader, net, num_steps)
            print(f"Iteration {counter}, Test Acc: {test_acc * 100:.2f}%\n")
            test_acc_hist.append(test_acc.item())

        counter += 1

torch.Size([50, 16, 4])
Iteration 0, Test Acc: 0.00%

torch.Size([50, 16, 4])
torch.Size([50, 16, 4])
torch.Size([50, 16, 4])
torch.Size([50, 16, 4])
torch.Size([50, 16, 4])
torch.Size([50, 16, 4])
torch.Size([50, 16, 4])
torch.Size([50, 16, 4])
torch.Size([50, 16, 4])
torch.Size([50, 16, 4])
torch.Size([50, 16, 4])
torch.Size([50, 11, 4])


In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-2, betas=(0.9, 0.999))
num_epochs = 10
loss_hist = []
test_acc_hist = []
counter = 0
train_loader = dataloaders['train']
test_loader = dataloaders['val']
scheduler = lr_scheduler.MultiStepLR(optimizer,
                        milestones=[8, 24, 28], # List of epoch indices
                        gamma =0.5) # Multiplicative factor of learning rate decay
net.to(device)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    since = time.time()
    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in tqdm(dataloaders[phase],leave=False):
                    try:
                      inputs = inputs.to(device)
                      labels = labels.to(device)

                      # zero the parameter gradients
                      optimizer.zero_grad()

                      # forward
                      # track history if only in train
                      with torch.set_grad_enabled(phase == 'train'):
                          outputs = model(inputs)
                          _, preds = torch.max(outputs, 1)
                          loss = criterion(outputs, labels)

                          # backward + optimize only if in training phase
                          if phase == 'train':
                              loss.backward()
                              optimizer.step()
                    except Exception as e:
                      pass
                    # statistics
                    print(labels.data)
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights

        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [ ]:
magdom_model = train_model(net,loss_fn,optimizer,scheduler)

In [ ]:
from snntorch import spikegen

# Iterate through minibatches
data = iter(dataloader_train)
data_it, targets_it = next(data)

# Spiking Data
spike_data = spikegen.rate(data_it, num_steps=num_steps)

In [ ]:
import matplotlib.pyplot as plt
import snntorch.spikeplot as splt
from IPython.display import HTML


In [ ]:
spike_data_sample = spike_data[:, 0, 0]

In [ ]:
fig, ax = plt.subplots()
anim = splt.animator(spike_data_sample, fig, ax)
# plt.rcParams['animation.ffmpeg_path'] = 'C:\\path\\to\\your\\ffmpeg.exe'

HTML(anim.to_html5_video())

#**Crop Images**

In [ ]:
import cv2 as cv
import os

#First get the list of images in the folder
source = "/content/drive/MyDrive/SNNTorch/"
list_of_original_images = os.listdir("/content/drive/MyDrive/SNNTorch") #Add the path to the folder
#Create a new directory to store the cropped images
imgpath = "/content/drive/MyDrive/CroppedImages/"

#Iterate through the image_list
for image_path in list_of_original_images:
    image_array = cv.imread(source + image_path) # Here we load image using opencv
    image_array = np.asarray(image_array) # Here we convert the image to numpy array
    print
    print(image_array.shape)
    height,width,channel = image_array.shape #Here we get the height,width and color channel
    cropped_image = image_array[9:458,60:518] # Using array slicing we cut some part of the image and save in a new variable

    # Write cropped image to Cropped Images folder
    cv.imwrite(imgpath + image_path, cropped_image)

# **Change name to better format**

In [ ]:
#First get the list of images in the folder
source = "/content/drive/MyDrive/SNNTorch/"
list_of_original_images = os.listdir("/content/drive/MyDrive/SNNTorch") #Add the path to the folder
#Create a new directory to store the cropped images
imgpath = "/content/drive/MyDrive/CroppedImages/"

#Iterate through the image_list
for image_path in list_of_original_images:
  list_val = image_path.split('-')
  Ms = list_val[1]
  Aex = list_val[3] + '-11'
  keff = list_val[6]
  alpha = list_val[8].split('.')
  alpha = alpha[0]+'.'+alpha[1]
  name = Ms + '_' + Aex + '_' + keff + '_' + alpha + '_' + '.png'
  os.rename(imgpath + image_path, imgpath + name)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/CroppedImages/Ms-1000000.0-Aex-1.5e-11-keff-100000.0-alpha-0.95.png' -> '/content/drive/MyDrive/CroppedImages/1000000.0_1.5e-11_100000.0_0.95_.png'